In [ ]:

    ################## Kreuzvaldierung der Modelle  ##################
from sklearn.model_selection import LeaveOneOut
import pandas as pd
import numpy as np

df = pd.read_csv("Results1.txt", sep='\t')  # import Feature-Liste
nnList = []
lmList = []
bpList = []

loo = LeaveOneOut()
print("Train", "||", "Test")
for train, test in loo.split(df):
    import pandas as pd
    import numpy as np
    df = pd.read_csv("Results1.txt", sep='\t')
    
    Train = df.iloc[train]
    Test = df.iloc[test]
    print(test)  
   


    import numpy as np
    import torch
    import torch.nn as nn
    import torch.nn.functional as F
    import torchvision
    import torchvision.transforms as transforms
    from tqdm.notebook import tqdm
    import pandas as pd
    import matplotlib.pyplot as plt
    from ipywidgets import IntProgress


    


    ################## Neural Network  ##################
    # Shuffle the rows of the dataframe
    # df = df.sample(frac=1)

    # # Calculate the number of rows in the dataframe
    # n_rows = df.shape[0]

    # # Split the dataframe into two dataframes
    # test = df.iloc[:int(n_rows * 0.2)]
    # Train = df.iloc[int(n_rows * 0.2):]



    class NN(nn.Module):
        def __init__(self, input_size, hidden_dim):
            super(NN, self).__init__()
            self.fc1 = nn.Linear(input_size, hidden_dim)
            self.fcH2 = nn.Linear(hidden_dim,hidden_dim)
            self.fcH3 = nn.Linear(hidden_dim,hidden_dim)
            self.fcH4 = nn.Linear(hidden_dim,hidden_dim)
            self.fcH5 = nn.Linear(hidden_dim,hidden_dim)
            self.fcH6 = nn.Linear(hidden_dim,hidden_dim)
            self.fc2 = nn.Linear(hidden_dim, 1)

        def forward(self, x):
            x = F.relu(self.fc1(x))
            x = F.relu(self.fcH2(x))
            x = F.relu(self.fcH3(x))
            x = F.relu(self.fcH4(x))
            x = F.relu(self.fcH5(x))
            x = F.relu(self.fcH6(x))
            x = self.fc2(x)
            return x

    # Model with 15 hidden layers and 8 features or input variables PTT, PTTs, .. and gender
    model = NN(hidden_dim=50, input_size=12)

    # Convert dataframes to tensors and change data type from float64 to float32
    X_test = Test[['PTTp', 'PTTs', 'PTTv', 'HR', 'height', 'weight', 'age', 'gender_female', 'gender_male','activity_run', 'activity_sit', 'activity_walk']].values
    X_test = X_test.astype(np.float32)
    y_test = Test['BP_dia'].values
    y_test = y_test.astype(np.float32)

    X_test = torch.tensor(X_test)
    y_test = torch.tensor(y_test)

    X = Train[['PTTp', 'PTTs', 'PTTv', 'HR', 'height', 'weight', 'age', 'gender_female', 'gender_male','activity_run', 'activity_sit', 'activity_walk']].values
    X = X.astype(np.float32)
    y = Train['BP_dia'].values
    y = y.astype(np.float32)

    X = torch.tensor(X)
    y = torch.tensor(y)




    def train(model, X, y, lr = 1e-3, weight_decay = 5e-4, epochs = 1000, display_step = 10):
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
        criterion = nn.MSELoss()

        for it in tqdm(range(epochs), desc='Training...'):
            model.train()
            output = model(X) 
            output = output.reshape(-1)
            # calculating training loss
            loss_train = criterion(output, y)
            

            optimizer.zero_grad()
            loss_train.backward()
            optimizer.step()


            #if display_step > 0 and it % display_step == 0:
                #print(f'Epoch {it:4}: loss_train: {loss_train.item():.5f}')

        return loss_train

    train(model, X, y)


    ################## Linear Model  ################## 
    from statsmodels.formula.api import ols
    lm_train = ols('BP_dia ~ PTTp + PTTs + PTTv + HR + gender_female + gender_male + height + weight + age + activity_run + activity_sit + activity_walk', data=Train).fit()
    # Generate predictions from the model
    predictions = lm_train.predict(Test)
    predictions = predictions.to_frame()



    import statistics as st
    nn1 = pd.DataFrame(model(X_test).detach().numpy())
    nn1 = nn1[0]


    bp = Test['BP_dia']
    bp.reset_index(inplace=True, drop=True)

    lm1 = predictions[0]
    lm1.reset_index(inplace=True, drop=True)

    nn_res = (nn1-bp)
    lm_res = (lm1-bp)
